In [1]:
import pandas as pd

# 1. Carregar os arquivos
# Usamos 'on_bad_lines' para pular linhas com erro se houver, e encoding utf-8
df_docvirt = pd.read_csv('dados_docvirt.csv')
df_extraidos = pd.read_csv('dados_extraidos.csv')

# 2. Ajustar 'dados_extraidos' (Limpeza e Padronização)
# O arquivo extraído tem muitas colunas vazias no final, vamos pegar apenas as úteis
cols_uteis = ['titulo', 'conteudo']
df_extraidos = df_extraidos[cols_uteis].copy()

# Adicionar as colunas que faltam para ficar no padrão do docvirt
# Se não houver informação, deixamos como None (NaN) ou um texto padrão
df_extraidos['pagina'] = None 
df_extraidos['status'] = 'extraido_novo' # Marcador para você saber a origem depois

# Reordenar as colunas do extraído para ficarem na mesma ordem do docvirt
colunas_ordem = df_docvirt.columns.tolist()
# Garante que o df_extraidos tenha todas as colunas necessárias
for col in colunas_ordem:
    if col not in df_extraidos.columns:
        df_extraidos[col] = None
        
df_extraidos = df_extraidos[colunas_ordem]

# 3. Juntar e Completar
# Concatenamos os dois. Colocamos o docvirt primeiro, pois ele parece mais completo (tem paginação)
df_final = pd.concat([df_docvirt, df_extraidos], ignore_index=True)

# 4. Remover duplicatas
# Se o mesmo 'titulo' existir nos dois, mantemos o primeiro (que veio do docvirt)
# Isso garante que completamos a lista sem criar linhas repetidas
df_final = df_final.drop_duplicates(subset=['titulo'], keep='first')

# 5. Visualizar e Salvar
print(f"Total de linhas no DocVirt original: {len(df_docvirt)}")
print(f"Total de linhas no Extraídos original: {len(df_extraidos)}")
print(f"Total de linhas após juntar e limpar: {len(df_final)}")

# Exibe as primeiras linhas
display(df_final.head())

# Salva o resultado final em um novo CSV
df_final.to_csv('dados_consolidados_completo.csv', index=False)
print("Arquivo 'dados_consolidados_completo.csv' salvo com sucesso!")

Total de linhas no DocVirt original: 9126
Total de linhas no Extraídos original: 9126
Total de linhas após juntar e limpar: 5803


,pagina,titulo,conteudo,status
0,1,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\nNUM_ANTDOC: 1\nPAGINAÇÃO: \nTIPO...,sucesso
1,2,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\nNUM_ANTDOC: 2\nPAGINAÇÃO: \nTIPO...,sucesso
2,3,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\nNUM_ANTDOC: 3\nPAGINAÇÃO: \nTIPO...,sucesso
3,4,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\nNUM_ANTDOC: 4\nPAGINAÇÃO: \nTIPO...,sucesso
4,5,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\nNUM_ANTDOC: 5\nPAGINAÇÃO: \nTIPO...,sucesso


Arquivo 'dados_consolidados_completo.csv' salvo com sucesso!


In [6]:
# 1. Identificar onde o conteúdo está vazio ou nulo
mask_sem_conteudo = (
    df_final['conteudo'].isna() | 
    (df_final['conteudo'].astype(str).str.strip() == '') | 
    (df_final['conteudo'].astype(str).str.lower() == 'nan')
)

df_manual = df_final[mask_sem_conteudo].copy()

# 2. Limpeza Crítica: Remover linhas que não têm 'titulo' (ID)
# Se não tem título, é erro de leitura do arquivo original e impossível de você buscar manualmente.
df_manual = df_manual.dropna(subset=['titulo'])

# 3. Selecionar apenas as colunas úteis para sua busca manual
# Mantive 'pagina' e 'status' para ajudar você a achar o documento no site/pasta
colunas_para_exportar = ['titulo', 'pagina', 'status']
df_manual = df_manual[colunas_para_exportar]

# 4. Criar uma coluna vazia para você preencher
df_manual['conteudo_manual'] = ''

# Mostra quantos sobraram
print(f"Você tem {len(df_manual)} documentos reais para buscar e preencher manualmente.")
display(df_manual.head())

# 5. Salvar para você trabalhar
df_manual.to_csv('para_preencher_manual.csv', index=False)
print("Arquivo 'para_preencher_manual.csv' gerado com sucesso.")

Você tem 1671 documentos reais para buscar e preencher manualmente.


,titulo,pagina,status,conteudo_manual
5360,(CO) Consulta Correspondência\CO_3458,5361,vazio_ou_imagem,
5362,(CO) Consulta Correspondência\CO_3460,5363,vazio_ou_imagem,
5363,(CO) Consulta Correspondência\CO_3461,5364,vazio_ou_imagem,
5364,(CO) Consulta Correspondência\CO_3462,5365,vazio_ou_imagem,
5366,(CO) Consulta Correspondência\CO_3464,5367,vazio_ou_imagem,


Arquivo 'para_preencher_manual.csv' gerado com sucesso.


In [7]:
# 1. Carregar os arquivos
# Usamos 'on_bad_lines' para pular linhas com erro se houver, e encoding utf-8
df_docvirt = pd.read_csv('dados_consolidados_completo.csv')
df_extraidos = pd.read_csv('dados_extraidos2.csv')

# 2. Ajustar 'dados_extraidos' (Limpeza e Padronização)
# O arquivo extraído tem muitas colunas vazias no final, vamos pegar apenas as úteis
cols_uteis = ['titulo', 'conteudo']
df_extraidos = df_extraidos[cols_uteis].copy()

# Adicionar as colunas que faltam para ficar no padrão do docvirt
# Se não houver informação, deixamos como None (NaN) ou um texto padrão
df_extraidos['pagina'] = None 
df_extraidos['status'] = 'extraido_novo' # Marcador para você saber a origem depois

# Reordenar as colunas do extraído para ficarem na mesma ordem do docvirt
colunas_ordem = df_docvirt.columns.tolist()
# Garante que o df_extraidos tenha todas as colunas necessárias
for col in colunas_ordem:
    if col not in df_extraidos.columns:
        df_extraidos[col] = None
        
df_extraidos = df_extraidos[colunas_ordem]

# 3. Juntar e Completar
# Concatenamos os dois. Colocamos o docvirt primeiro, pois ele parece mais completo (tem paginação)
df_final = pd.concat([df_docvirt, df_extraidos], ignore_index=True)

# 4. Remover duplicatas
# Se o mesmo 'titulo' existir nos dois, mantemos o primeiro (que veio do docvirt)
# Isso garante que completamos a lista sem criar linhas repetidas
df_final = df_final.drop_duplicates(subset=['titulo'], keep='first')

# 5. Visualizar e Salvar
print(f"Total de linhas no DocVirt original: {len(df_docvirt)}")
print(f"Total de linhas no Extraídos original: {len(df_extraidos)}")
print(f"Total de linhas após juntar e limpar: {len(df_final)}")

# Exibe as primeiras linhas
display(df_final.head())

# Salva o resultado final em um novo CSV
df_final.to_csv('dados_consolidados_completo2.csv', index=False)
print("Arquivo 'dados_consolidados_completo2.csv' salvo com sucesso!")

Total de linhas no DocVirt original: 5803
Total de linhas no Extraídos original: 856
Total de linhas após juntar e limpar: 5803


/tmp/ipykernel_7606/2005690329.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_docvirt, df_extraidos], ignore_index=True)


,pagina,titulo,conteudo,status
0,1.0,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\nNUM_ANTDOC: 1\nPAGINAÇÃO: \nTIPO...,sucesso
1,2.0,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\nNUM_ANTDOC: 2\nPAGINAÇÃO: \nTIPO...,sucesso
2,3.0,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\nNUM_ANTDOC: 3\nPAGINAÇÃO: \nTIPO...,sucesso
3,4.0,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\nNUM_ANTDOC: 4\nPAGINAÇÃO: \nTIPO...,sucesso
4,5.0,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\nNUM_ANTDOC: 5\nPAGINAÇÃO: \nTIPO...,sucesso


Arquivo 'dados_consolidados_completo2.csv' salvo com sucesso!


# 1. Importação e Carga dos Dados
Nesta etapa, carregamos o arquivo CSV consolidado. Usamos o parâmetro `keep_default_na=False` e `na_values=['']` para ter um controle melhor sobre o que é considerado vazio.

In [1]:
import pandas as pd
import re

# Carregar o dataset
arquivo = 'dados_consolidados_completo2.csv'

# Lemos o arquivo.
# O encoding='utf-8' é padrão, mas garantimos para evitar erro de acentuação.
df = pd.read_csv(arquivo)

# Visualizar a estrutura inicial
print(f"Linhas: {df.shape[0]} | Colunas: {df.shape[1]}")
display(df.head())

Linhas: 5803 | Colunas: 4


,pagina,titulo,conteudo,status
0,1.0,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\nNUM_ANTDOC: 1\nPAGINAÇÃO: \nTIPO...,sucesso
1,2.0,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\nNUM_ANTDOC: 2\nPAGINAÇÃO: \nTIPO...,sucesso
2,3.0,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\nNUM_ANTDOC: 3\nPAGINAÇÃO: \nTIPO...,sucesso
3,4.0,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\nNUM_ANTDOC: 4\nPAGINAÇÃO: \nTIPO...,sucesso
4,5.0,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\nNUM_ANTDOC: 5\nPAGINAÇÃO: \nTIPO...,sucesso


# 2. Tratamento de Tipos e Limpeza Básica
Aqui fazemos a normalização dos dados existentes:
1. **Página:** Convertemos de Float (1.0) para Inteiro (1), mantendo `<NA>` onde não houver número.
2. **Textos:** Removemos espaços em branco extras no início e fim das strings (trimming).
3. **IDs:** Criamos uma coluna limpa apenas com o código do documento (ex: CO_0001) para facilitar buscas futuras.

In [2]:
# 1. Ajustar a coluna 'pagina' para números inteiros (Int64 aceita nulos)
# Isso remove o ".0" visual (ex: 5.0 vira 5)
df['pagina'] = pd.to_numeric(df['pagina'], errors='coerce').astype('Int64')

# 2. Limpeza de espaços em branco (strip) em todas as colunas de texto
# Isso evita que " Texto" seja diferente de "Texto"
cols_texto = df.select_dtypes(include=['object']).columns
for col in cols_texto:
    df[col] = df[col].astype(str).str.strip()
    # Corrige onde virou string 'nan' ou 'None' para valor nulo real
    df[col] = df[col].replace({'nan': None, 'None': None, '': None})

# 3. Extrair um ID limpo do título
# O título atual é longo (ex: "(CO) Consulta...\\CO_0001"). Vamos criar uma coluna 'id_doc'
# Regex pega tudo que vem depois da última barra invertida
df['id_doc'] = df['titulo'].astype(str).str.extract(r'\\([^\\]+)$')

display(df[['id_doc', 'pagina', 'titulo']].head())

,id_doc,pagina,titulo
0,CO_0001,1,(CO) Consulta Correspondência\CO_0001
1,CO_0002,2,(CO) Consulta Correspondência\CO_0002
2,CO_0003,3,(CO) Consulta Correspondência\CO_0003
3,CO_0004,4,(CO) Consulta Correspondência\CO_0004
4,CO_0005,5,(CO) Consulta Correspondência\CO_0005


# 3. Extração de Metadados (Enriquecimento)
A coluna `conteudo` possui muitas informações aglutinadas (Remetente, Destinatário, Data, Resumo).
Vamos usar **Expressões Regulares (Regex)** para extrair esses dados para novas colunas, **sem apagar** a coluna `conteudo` original.

Isso permitirá que você filtre por data ou remetente depois, o que é impossível com o texto puro.

In [3]:
# Função para extrair padrões do texto cru
def extrair_campo(texto, padrao):
    if pd.isna(texto):
        return None
    match = re.search(padrao, texto, re.MULTILINE | re.IGNORECASE)
    return match.group(1).strip() if match else None

# Padrões de regex baseados no formato do seu arquivo
padroes = {
    'remetente_extraido': r'REMETENTE:\s*(.*?)(?:\n|$)',
    'destinatario_extraido': r'DESTINATARIO:\s*(.*?)(?:\n|$)',
    'data_extraida': r'DATA_INICIO:\s*(.*?)(?:\n|$)',
    'tipo_doc_extraido': r'TIPO CORRESPONDÊNCIA:\s*(.*?)(?:\n|$)'
}

# Aplica a extração para criar novas colunas
print("Extraindo metadados do conteúdo... isso pode levar alguns segundos.")

for nova_coluna, regex in padroes.items():
    df[nova_coluna] = df['conteudo'].apply(lambda x: extrair_campo(str(x), regex))

# Visualizar o resultado das novas colunas
colunas_visualizar = ['id_doc'] + list(padroes.keys())
display(df[colunas_visualizar].head())

Extraindo metadados do conteúdo... isso pode levar alguns segundos.


,id_doc,remetente_extraido,destinatario_extraido,data_extraida,tipo_doc_extraido
0,CO_0001,Iris Abbott,Candido Portinari,1940/11/08,Telegrama
1,CO_0002,Júlio Jorge Abeid Filho,Candido Portinari,1956/10/04,Carta
2,CO_0003,Lívio Abramo,Candido Portinari,1945/11/26,Carta
3,CO_0004,Tharcema Cunha de Abreu,Candido Portinari,1946/ /,Carta
4,CO_0005,Aníbal Freire,Júlio Prestes,1926/06/05,Carta


# 4. Análise de Qualidade e Exportação
Verificamos quantas linhas continuam sem conteúdo (para seu controle manual posterior) e salvamos o arquivo tratado.

In [4]:
# Contar quantos documentos ainda estão vazios
vazios = df['conteudo'].isna() | (df['conteudo'] == '')
print(f"Total de documentos tratados: {len(df)}")
print(f"Total de documentos com conteúdo vazio (para preenchimento manual): {vazios.sum()}")

# Reordenar colunas para deixar as mais importantes no início, mas mantendo TODAS
cols_iniciais = ['id_doc', 'pagina', 'status', 'data_extraida', 'remetente_extraido', 'destinatario_extraido']
# Pega o resto das colunas que não estão na lista de iniciais
cols_restantes = [c for c in df.columns if c not in cols_iniciais]
df_final = df[cols_iniciais + cols_restantes]

# Salvar
arquivo_saida = 'dados_tratados_final.csv'
df_final.to_csv(arquivo_saida, index=False)

print(f"Arquivo '{arquivo_saida}' gerado com sucesso!")

Total de documentos tratados: 5803
Total de documentos com conteúdo vazio (para preenchimento manual): 1672
Arquivo 'dados_tratados_final.csv' gerado com sucesso!
